# AIC and p-Hacking

## Table of Contents

[Introduction](#intro)<br>
1 - [The Akaike Information Criterion](#aic)<br>
2 - [p-Values and p-Hacking](#p-hack)<br>

## Introduction<a id='intro'></a>

In this notebook we will introduce the Akaike Information Criterion ($\text{AIC}$) and give an example of p-Hacking. $\text{AIC}$ is a way of assessing the quality of a model given a set of data, an important concept for checking model performance. The target audience of this notebook was such that there was no need for calculus based statistics, and hence, no direct calculations of liklihood functions. Instead, we focus on providing intuition and understanding of $\text{AIC}$ rather then calculation. Our section in p-Values and p-Hacking involves a brief review of p-Values and provides an example of modifying your results, creating a bias known as p-Hacking. p-Hacking is very common in frequetist statistics today, and it's important to identify and be aware of the potential pitfalls that exists in data analysis.

The data for this notebook come from a study looking at NO$_2$ levels in the United States ([Novotny et al ES&T, 2011](https://pubs.acs.org/doi/abs/10.1021/es103578x)). The US Environmental Protection Agency collected this data in 2006 (known as *allmodelbuildingdata*) and the authors of this paper selected a handful of features in order to give create a positively correlated regression model (known as *final_model*). We'll be using both datasets in this notebook.

This notebook has been created as a small introduction to $\text{AIC}$ and p-Hacking, bordering an interactie notebook and a homework assignment. This notebook was created in part for [ER-190C, a course taught in Fall 2018 at UC Berkeley](https://github.com/ds-modules/ER-190C). What is presented here is but a sample of the completed homework assignment for the class. All materials (explainations, problem, solutions, etc.), except the data, were created by me.

### Imports

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## 1. The Akaike Information Criterion<a id='aic'></a>

The Akaike Information Criterion ([$\text{AIC}$](https://www.youtube.com/watch?v=Nco_kh8xJDs)) assess the ***quality*** of a model given a set of data. Depending on the data that we use in our model, in this case, the features we add, AIC may be used to tell us how our model performs with the data given. Sometimes adding more data (features) improves the quality, sometimes less. Other times adding the right features may improve the quality.

$\text{AIC}$ is important because we can use it as a form of model selection. **Our goal is to find a model that has the highest *quality* given a list of models.** The higher the quality, the better our model performs and the more desirable it is. Your job in this section is to add features to *final_model* from *allmodelbulidingdata* and assess whether adding specific features improves the model or not. This may seem daunting, but we'll guide you in this process.

Now that you have some information about what AIC is doing and why it's important, we define $\text{AIC}$ as the following:

$\text{AIC} = 2 \times (\text{number of features}) - 2 \times \log(\text{maximum value of likelihood function})$

where $\log$ is $\ln$.  The smaller $\text{AIC}$ is, the greater the model performs (lower test error). A likelihood function is a statistical topic that we won't go into, but we'll provide the code on how to implement it.

The way to interpret an $\text{AIC}$ number is that smaller it is, the greater the model performs.

First, let's load and clean both datasets, *final_model* and *allmodelbuildingdata*, so that we may be able to experiment with implementing models with different features.

In [24]:
#Loading allmodelbuildingdata, no cleaning necessary since 
NO2_all = pd.read_csv('BechleLUR_2006_allmodelbuildingdata.csv')


#Loading final_model daya
NO2 = pd.read_csv("BechleLUR_2006_finalmodel.csv")

#Cleaning data, removing categorical and non-predictive variables
NO2.drop(columns = 
         ["Latitude", "Longitude", "State", "Predicted_NO2_ppb", 
            "Location_type", "Monitor_ID"], inplace = True)


We have our dataset, now we need a function to produce our maximum value of our liklihood function. Let's import what we need for $\text{AIC}$. However, before we do that we must add a column of zeros to our model as it's a multiple regression model.

In [28]:
#Add column of zeros to adjust for constants
#This is needed in regression models
NO2["Constant"] = np.ones(len(NO2))

#Import to find maximum value of log likelihood
import statsmodels.regression.linear_model as sm

    
#Function to find log likelihood and number of features
def likandfeatures(X, y):
    '''
    Description: A function created to find the maximum value of the log-likelihood function in an OLS
    model. Also returns the number of features in the likelihood
    
    Inputs: X, An NxN pandas dataframe that contains your features
            y, An Nx1 pandas dataframe that you're trying to fit with X
            
    Returns: A tuple containing:
               The amount of features in X when finding the log-likelihood function
               The maximum value of the log-likelihood function
    '''
    #Adding a column (feature) of ones to consider constants because
    #Statsmodel does not support constants, so we must manually add one.
    #Thus, features = number of columns in X + 1
    X_final = X.assign(Constant = np.ones(len(y)))
    
    return (sm.OLS(y, X_final).fit().llf, len(X_final.columns))


#Running Model. This finds the Log-Likelihood function
log_L = sm.OLS(NO2_all["Population_800"], NO2).fit().llf
log_L
#-3270.3083800895165

-3270.3083800895165

**Question 1.1** Defining the $\text{AIC}$ function outlined above.

In [ ]:
def aic(features, log_lik):
    '''
    A function that calculates the Akaike Information Criterion given the number of
    features and maximum value of our liklihood function.
    
    Args:
        features: int, num of features
        log_lik: float, maximum value of our log liklihood function
    Returns:
        Float, a solution to the formula. The lesser the number the greater the model.
    '''
       return 2*(features - log_lik)

Now that we have created a function to get our $\text{AIC}$, we now run several tests to see if it works. We use statsmodels to confirm that our $\text{AIC}$ is correct. 

**Question 1.2** Look at the table produced by statsmodels below. It should show a lot of information, including $\text{AIC}$. How close is our homemade $\text{AIC}$ with statsmodels? The fact that our $\text{AIC}$ is negative is simply a byproduct of our formula and doesn't have any meaning.

*Your Answer Here*

In [36]:
#Homemade implementation
homeAIC = likandfeatures(NO2, NO2_all['Population_800'])
print("This is the log-likelihood max value", homeAIC[1])
print("This is the aic:", aic(*homeAIC))

#Statsmodels implementation
sm.OLS(NO2_all["Population_800"], NO2.assign(Constant = np.ones(len(NO2_all)))).fit().summary()

This is the log-likelihood max value 8
This is the aic: -6556.616760179033


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         Population_800   R-squared:                       0.447
Model:                            OLS   Adj. R-squared:                  0.436
Method:                 Least Squares   F-statistic:                     41.62
Date:                Wed, 09 Jan 2019   Prob (F-statistic):           7.65e-43
Time:                        16:03:40   Log-Likelihood:                -3270.3
No. Observations:                 369   AIC:                             6557.
Df Residuals:                     361   BIC:                             6588.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Observed_NO2_ppb          22.9083     29.185      0.785      0.433     -34.486      80.302
WRF+DOMINO               110.6330     29.365      3.768      0.000      52.885     168.381
800m_Impervious_%         28.7800      5.853      4.917      0.000      17.269      40.291
Elevation_truncated_km -2107.6038   1008.378     -2.090      0.037   -4090.636    -124.572
800m_MajorRoads_km       179.3514     48.135      3.726      0.000      84.691     274.011
100m_MinorRoads_km       813.2976    594.456      1.368      0.172    -355.734    1982.330
Distance_to_coast_km       0.1480      0.220      0.674      0.501      -0.284       0.580
Constant                -609.8006    244.238     -2.497      0.013   -1090.108    -129.493
==============================================================================
Omnibus:                      484.991   Durbin-Watson:                   1.742
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            62363.535
Skew:                           6.209   Prob(JB):                         0.00
Kurtosis:                      65.466   Cond. No.                     7.73e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.73e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**Question 1.3** Now look at this statsmodels table. How does it compare with our homemade model? Is it greater, or less than the previous feature used (`Population_800`). Why do you think the feature `Major_400` produces the results that it does? *Hint*: Look at the features in `NO2`. Could any of these features be colinear with `Population_800`? How about `Major_400`?

*Your Answer Here*

In [34]:
#Homemade implementation
homeAIC = likandfeatures(NO2, NO2_all["Major_400"])
print("This is the log-likelihood max value", homeAIC[1])
print("This is the aic:", aic(*homeAIC))

#Statsmodels implementation
sm.OLS(NO2_all["Major_400"], NO2.assign(Constant = np.ones(len(NO2_all)))).fit().summary()

This is the log-likelihood max value 8
This is the aic: -1043.2092697974288


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Major_400   R-squared:                       0.414
Model:                            OLS   Adj. R-squared:                  0.402
Method:                 Least Squares   F-statistic:                     36.39
Date:                Wed, 09 Jan 2019   Prob (F-statistic):           2.10e-38
Time:                        15:53:12   Log-Likelihood:                -513.60
No. Observations:                 369   AIC:                             1043.
Df Residuals:                     361   BIC:                             1074.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Observed_NO2_ppb           0.0541      0.017      3.253      0.001       0.021       0.087
WRF+DOMINO                -0.0417      0.017     -2.494      0.013      -0.075      -0.009
800m_Impervious_%          0.0041      0.003      1.221      0.223      -0.002       0.011
Elevation_truncated_km    -1.3624      0.574     -2.372      0.018      -2.492      -0.233
800m_MajorRoads_km         0.3019      0.027     11.014      0.000       0.248       0.356
100m_MinorRoads_km        -0.9444      0.339     -2.790      0.006      -1.610      -0.279
Distance_to_coast_km       0.0002      0.000      1.520      0.129   -5.58e-05       0.000
Constant                  -0.0720      0.139     -0.517      0.605      -0.345       0.202
==============================================================================
Omnibus:                      436.659   Durbin-Watson:                   1.939
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            32220.926
Skew:                           5.357   Prob(JB):                         0.00
Kurtosis:                      47.507   Cond. No.                     7.73e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.73e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**Question 1.4** How does this compare to the past two features? Why do you think it produced the results that it did relative to the past two features?

*Your Answer Here*

In [35]:
#Homemade implementation
homeAIC = likandfeatures(NO2, NO2_all["Major_1200"])
print("This is the log-likelihood max value", homeAIC[1])
print("This is the aic:", aic(*homeAIC))

#Statsmodels implementation
sm.OLS(NO2_all["Major_1200"], NO2.assign(Constant = np.ones(len(NO2_all)))).fit().summary()

This is the log-likelihood max value 8
This is the aic: -2094.8362071539236


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Major_1200   R-squared:                       0.609
Model:                            OLS   Adj. R-squared:                  0.601
Method:                 Least Squares   F-statistic:                     80.28
Date:                Wed, 09 Jan 2019   Prob (F-statistic):           1.02e-69
Time:                        15:54:27   Log-Likelihood:                -1039.4
No. Observations:                 369   AIC:                             2095.
Df Residuals:                     361   BIC:                             2126.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Observed_NO2_ppb           0.2352      0.069      3.403      0.001       0.099       0.371
WRF+DOMINO                -0.1135      0.070     -1.633      0.103      -0.250       0.023
800m_Impervious_%          0.0657      0.014      4.741      0.000       0.038       0.093
Elevation_truncated_km    -3.2981      2.388     -1.381      0.168      -7.994       1.397
800m_MajorRoads_km         1.5682      0.114     13.759      0.000       1.344       1.792
100m_MinorRoads_km        -2.0478      1.408     -1.455      0.147      -4.816       0.720
Distance_to_coast_km      -0.0003      0.001     -0.502      0.616      -0.001       0.001
Constant                  -0.0858      0.578     -0.148      0.882      -1.223       1.052
==============================================================================
Omnibus:                      153.059   Durbin-Watson:                   2.110
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              656.776
Skew:                           1.784   Prob(JB):                    2.42e-143
Kurtosis:                       8.476   Cond. No.                     7.73e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.73e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

---

## 2. p-Values and p-Hacking<a id='p-hack'></a>

For this problem, we'll load NO2 data and use the package Statsmodels to help us calculate p-Values. We'll do some analysis on our results and discuss a potential danger/caution when making conclusions based on p-Values: p-Hacking. We assume some familarity with p-Values, however, we've taken the liberty of reviewing it here (as from experience, understanding p-Values can be sometimes tricky).

A p-value (taken from the [Data 8 textbook](https://www.inferentialthinking.com/chapters/11/3/Decisions_and_Uncertainty)) is the chance, based on the model in the null hypothesis, that the test statistic is equal to the value that was observed in the data or is even further in the direction of the alternative. Informally, if a p-value is really low, then the chance of your observation happening is low and may not be probable.

We'll now create a multiple regression model and find it's p-values by using the package Statsmodels. We'll load data from the final data at a study looking at NO$_2$ levels in the United States ([Novotny et al ES&T, 2011](https://pubs.acs.org/doi/abs/10.1021/es103578x)).

In [4]:
#Loading data
NO2 = pd.read_csv("BechleLUR_2006_finalmodel.csv")

#Cleaning data, removing categorical and non-predictive variables
NO2.drop(columns = 
         ["Latitude", "Longitude", "State", "Predicted_NO2_ppb", 
            "Location_type", "Monitor_ID"], inplace = True)

NO2.head(3)

,Observed_NO2_ppb,WRF+DOMINO,800m_Impervious_%,Elevation_truncated_km,800m_MajorRoads_km,100m_MinorRoads_km,Distance_to_coast_km
0,23.884706,11.615223,58.9488,0.304,1.35858,0.61637,313.0
1,25.089886,11.472677,71.4093,0.304,1.55566,0.26126,323.8
2,19.281969,8.990372,53.5480,0.304,1.59508,0.39460,308.4


Here we'll use "Observed_NO2_ppb" as our response variable and the rest of the variables we'll label as predictors. Then we'll run a multiple regression model.

In [5]:
#Setting variables 
X = NO2.drop("Observed_NO2_ppb", axis=1)
Y = NO2["Observed_NO2_ppb"]

In [6]:
import statsmodels.api as sm

# In order to have an intercept, we need to add a column of 1's to X
# This is needed in regression models
X2 = sm.add_constant(X)

#Running multiple regression
sm_model = sm.OLS(Y, X2)
results = sm_model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:       Observed_NO2_ppb   R-squared:                       0.778
Model:                            OLS   Adj. R-squared:                  0.775
Method:                 Least Squares   F-statistic:                     212.0
Date:                Tue, 08 Jan 2019   Prob (F-statistic):          3.36e-115
Time:                        15:10:09   Log-Likelihood:                -938.93
No. Observations:                 369   AIC:                             1892.
Df Residuals:                     362   BIC:                             1919.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      2

Statsmodels gives access to a ton of information; it may be a little daunting seeing it for the first time. However we may easily extract the p-Values from our regression model.

In [8]:
results.pvalues

const                     2.634006e-09
WRF+DOMINO                1.840009e-58
800m_Impervious_%         2.056259e-21
Elevation_truncated_km    1.950387e-09
800m_MajorRoads_km        1.865722e-04
100m_MinorRoads_km        1.998295e-02
Distance_to_coast_km      2.417248e-03
dtype: float64

In Statsmodels, the null hypothesis is defined as there being no statistically significant relationship between the term ($x$) and our prediction ($\hat{y}$). Rejecting the null hypothesis is dependent on the $\alpha$ level, the minimum percentage that you're willing to accept the null. The smaller your $\alpha$, the stricter your test. You'll be more confident at determining whether what you're measuring is likely due to chance or bias.

**Question 2.1:** Interpret the p-values for each of the seven variables in results. Determine whether there is a statistically significant relationship between each variable and your predicted variable. You are free to choose your own $\alpha$ value, whatever you feel is appropriate.

**Your answer here**
<br> $\alpha$ = ...
<br> const: ...
<br> WRF+DOMINO: ...
<br> 800m\_Impervious_%: ...
<br> Elevation_truncated_km: ...
<br> 800m_MajorRoads_km: ...
<br> 100m_MinorRoads_km: ...
<br> Distance_to_coast_km: ...

***Solution 1***
**<br> $\alpha$ = 0.05
<br> const: There is a statistically significant relationship between const and our prediction
<br> WRF+DOMINO: There is a statistically significant relationship between WRF+DOMINO and our prediction
<br> 800m\_Impervious_%: There is a statistically significant relationship between 800m\_Impervious_% and our prediction
<br> Elevation_truncated_km: There is a statistically significant relationship between Elevation_truncated_km and our prediction
<br> 800m_MajorRoads_km: There is a statistically significant relationship between 800m_MajorRoads_km and our prediction
<br> 100m_MinorRoads_km: There is no relationship between 100m_MinorRoads_km and our prediction
<br> Distance_to_coast_km: There is a statistically significant relationship between Distance_to_coast_km and our prediction**

***Solution 2***
**<br> $\alpha$ = 0.01
<br> const: There is a statistically significant relationship between const and our prediction
<br> WRF+DOMINO: There is a statistically significant relationship between WRF+DOMINO and our prediction
<br> 800m\_Impervious_%: There is a statistically significant relationship between 800m\_Impervious_% and our prediction
<br> Elevation_truncated_km: There is a statistically significant relationship between Elevation_truncated_km and our prediction
<br> 800m_MajorRoads_km: There is a statistically significant relationship between 800m_MajorRoads_km and our prediction
<br> 100m_MinorRoads_km: There is a statistically significant relationship between 100m_MinorRoads_km and our prediction
<br> Distance_to_coast_km: There is a statistically significant relationship between Distance_to_coast_km and our prediction**

Depending on your $\alpha$ level, some variables may be statistically significant. The bias associated with choosing a p-value to determine significance is called *p-hacking*. In this case, choosing a higher or lower $\alpha$ level as a result of seeing the p-values is subject to this bias (in other words, unless you have a standard go-to $\alpha$ level, you were p-hacking). It's often best practice to pick an $\alpha$ level *before* seeing your results.

In general, *p-hacking* is the act of analyzing data in order to find patterns that are statistically significant that in reality have no actual significance. One usually has a bias on whether to reject the null or not, often subconciouslly, but sometimes deliberate. It's easy to fall prey to this bias, and it's important to keep in mind that when doing these hypothesis testings. 

In creating `results`, we added an extra column of ones in order to fit our model properly. Let's dig a little deeper with `const`.

**Question 2.2:** Comment further on the statistical significance with the extra column of ones and your prediction. What does a column of ones represents? Are the ones truely in your data? Why doesn't it make sense or matter?

**Solution***

**A column of ones doesn't represent any of our original data. There can't be anything statistically significant about it with relation to our analysis/inference about our data. A column of ones is mearly to aid in our model fitting. It's meaningless to try to give it meaning because the column of ones is fabricated.**

A humorous example of this taken to an outragous level can be found on [Tyler Vigen's website](http://www.tylervigen.com/spurious-correlations). Pushing for significance between your variables without understanding their context leaves you subject to p-hacking.

---